# Classification with K nearest neighbors (kNN)

### 1. kNN Background

#### 1.1 Type of algorithm

Today, we learned that kNN is a supervised machine learning methods. It can be used for both classification and regression problems. This means that we are given a labelled dataset consiting of training observations $(x,y)$ and would like to capture the relationship between the features $x$ and the outcomes $y$. Here, we will use kNN for classification. This means that $y$ takes categorical values, and the task then is to predict the category $y$ for new observations of $x$. 

#### 1.2 Distance measure

In the classification setting, the k-nearest neighbor algorithm essentially boils down to forming a majority vote between the k most similar instances to a given “unseen” observation. Similarity is defined according to a distance metric between two data points. The k-nearest-neighbor classifier is commonly based on the Euclidean distance between a test sample and the specified training samples. Let $x^{(i)}$ be data point with $p$ features $(x^{(i)}_{1}, x^{(i)}_{2},..., x^{(i)}_{p})$, $N$ be the total number of data points $(i=1,2,...,N)$.  The Euclidean distance between $x^{(i)}$ and $x^{(j)}$ is defined as: 


$$d(x^{(i)}, x^{(j)}) = \sqrt{(x^{(i)}_{1} - x^{(j)}_{1})^2 + (x^{(i)}_{2} - x^{(j)}_{2})^2 + ... + (x^{(i)}_{p} - x^{(j)}_{p})^2}$$

Sometimes other measures can be more suitable for a given setting and include the Manhattan, Chebyshev and Hamming distance.

#### 1.3 Algorithm steps

The kNN algorithm consists of four simple steps:

    (1) Choose the number k of neighbors.

    (2) Take the k nearest neighbors of the new data point, according to your distance metric.

    (3) Among these k neighbors, count the number of data points to each category.

    (4) Assign the new data point to the category with the highest count among neighbours of the new data point.
    
Throughout the notebook we will use a number of libraries and functions. Check out their manuals to familiarize yourselves with the methods.

### 2. Importing and preperation of data

#### 2.1 Import libraries

Let's load some libraries we will use throughout the notebook.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp

%matplotlib inline

#### 2.2 Load dataset

We will analyse the Iris data set you have already familiarised yourself with earlier during the course. It includes 3 iris species with 50 samples each. For each sample, we have 4 features: sepal length and width as well as petal length and width, all in cm. Here, species is a categorical outcome that we aim to predict from the numeric features using KNN.

__TASK:__ Use ```pd.read_csv()``` to load the Iris data set in ```Iris.csv```, which located in the same folder as your notebook.

In [ ]:
# Importing the dataset


#### 2.3 Summarize the Dataset

At the start, it is always best to get a quick glimpse of the data and its format. We can get a quick idea of how many samples (rows) and how many attributes (columns) the data contains using ```shape()``` on the ```dataset``` object.

In [ ]:
# using shape to get a first idea of the data format


Using ```head(n)```  on the ```dataset``` object we can see the first $n$ lines of the data table.

In [ ]:
# Looking at the first 5 rows of the data table


Using describe(), we can get some summary statistics of the data. 

In [ ]:
# show summary statistics


Let’s now take a look at the number of instances (rows) that belong to each class. We can view this as an absolute count.

In [ ]:
dataset.groupby('Species').size()

#### 2.4 Dividing data into features and labels

As we can see dataset contain six columns: Id, SepalLengthCm, SepalWidthCm, PetalLengthCm, PetalWidthCm and Species. The actual features are described by columns 1-4. The last column contains labels of samples. Firstly we need to split data into two arrays: X (features) and y (labels).

In [ ]:
feature_columns = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm','PetalWidthCm']
X = dataset[feature_columns].values
y = dataset['Species'].values

# Alternative way of selecting features and labels arrays:
# X = dataset.iloc[:, 1:5].values
# y = dataset.iloc[:, 5].values

#### 2.5 Label encoding

Our species labels are categorical, encoded in character strings. The KNN function, KNeighborsClassifier, which we will use, does not accept string labels. We therefore use LabelEncoder to transform them into numbers: Iris-setosa correspond to 0, Iris-versicolor correspond to 1 and Iris-virginica correspond to 2.

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

#### 2.6 Spliting dataset into training set and test set

In machine learning, it is common to split datasets into training and test data sets. We will learn more about this during the next lecture. 

Essentially, splitting the data we can test the model performance on data that is independent from the training data. The training data is used to 'train' our machine learning model, i.e. to adjust the method's parameters, and the test dataset is then used to assess the performance of the model on previously unseen data using an appropriate metric of performance. 

Let's split dataset into training set and test set, to check later on whether or not our classifier works correctly.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

Note: Often, different features can take values on very different scales, in which case it would make sense to scale or 'normalise' data before further analysis. In our case, the different features, sepal and petal lengths and widths, take similar values, and we therefore don't have to worry about normalisation. Just keep in mind that in other data scenarios it can be extremly important to apply feature scaling before running classification algorithms.

### 3. Visual data exploration

In __§2.3__, we took an initial look at the data format. Equally important is looking at our data by means of different visualisations to get a feel of the data before starting any further model analysis via machine learning. In the following, we will plot the data in different ways. 

__TASK__: Think about which plots you find most helpful and why!

#### 3.1. Parallel Coordinates

Parallel coordinates is a plotting technique for plotting multivariate data. It allows one to see clusters in data and to estimate other statistics visually. Using parallel coordinates, single data points are represented as connected line segments with features along the x-axis and their values on the y-axis. Each connected line segment represents one data point, and we can colour the lines by the outcome label (species). Points that tend to cluster will appear closer together.

__TASK__: Using the ```parallel_coordinates()``` function from the ```pandas.plotting``` library, make a parallel coordinates plot with lines coloured by 'Species'. Tip: Make sure to remove the Id column using ```dataset.drop("Id", axis=1)``` instead of the full dataset. Make sure to label the axes and provide a legend for the species colours. 

In [ ]:
from pandas.plotting import parallel_coordinates

plt.figure(figsize=(15,10))

# use parallel_coordinates()

# add a title, axes labels and a legend


plt.show()

#### 3.2. Pairplot

In multivariate analysis we are concerned with the relations between various data variables. Feature variables may be statistically independent or display statistical dependence. To get an idea, it often helps to look at these dependencies beforehand, but visualisation in high dimensions are tricky, and 2- or at most 3-dimensional visualisations are more intuitive to us. 

Looking at pairwise dependencies is useful when you want to visualize the relationship between multiple variables separately within subsets of your dataset.

__TASK:__ Use the ```pairplot()``` function from seaborn to make pairwise scatterplots, grouped by 'Species', along with the 1-d marginal distributions of features. Again, make sure to drop the Id-column in dataset. 

In [ ]:
plt.figure()
# use pairplot()

plt.show()

# NOTE, there will be a UserWarning due to a bug in matplotlib - ignore it!

#### 3.3. Boxplots

Boxplots are a simple way to visualise 1-d distributions. 

__TASK:__ Make boxplots of the 4 features grouped by species. Tip: The function ```boxplot()``` is applied directly to your dataset object (with the Id column dropped).

In [ ]:
plt.figure()
# use boxplot()

plt.show()

#### 3.4. Alternatives to Boxplots

Box plots are not everyone's favourite, in fact, some journals have banned them. We can use __strip plots__ or __violine plots__ as an alternative. Why do you think people would prefer those over box plots?
Which of the three do you prefer and why?

#### 3.4.1. Strip plots

__TASK:__ Make a figure with four subplots, for each feature, and use the ```stripplot``` function from seaborn, grouping by species. Using the ```size``` property you can control the size of dots in the plots, which can be helpful to better see the data points. 

In [ ]:
# strip plots of the data
plt.figure(figsize=(10,10))
# creating four subplots, one for each feature, arranged in 2 rows and 2 columns
plt.subplot(2,2,1)
# use stripplot in each of the subplots

plt.subplot(2,2,2)

plt.subplot(2,2,3)

plt.subplot(2,2,4)

plt.show()

#### 3.4.1. Violin plots

__TASK:__ Make a figure with four subplots, for each feature, and use the ```violinplot``` function from seaborn, grouping by species. 

In [ ]:
# violin plots
plt.figure(figsize=(10,10))
plt.subplot(2,2,1)
# same as above for stripplot, now with violinplot()

plt.show()

#### 3.5. Other options for visualisation of high-dimensional data

Sometimes it is also useful to look at more than 2 variables together. We can use 3D plots to look at 3 variables together at a time, using color, shape, size and other properties of 3D and 2D objects. 

__Optional task:__ Have a go and use ```scatter()``` to make a 3D scatter plot using marker sizes to visualize the fourth dimenssion which is Petal Width [cm].

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(1, figsize=(20, 15))
ax = Axes3D(fig)
# create scatterplot, plotting X[:,i], for i = 0,1,2 and using the 4th feature X[:,3] as marker size

# adding class labels to the mean points in each class
for name, label in [('Virginica', 0), ('Setosa', 1), ('Versicolour', 2)]:
    ax.text3D(X[y == label, 0].mean(),
              X[y == label, 1].mean(),
              X[y == label, 2].mean(), name,
              horizontalalignment='center',
              bbox=dict(alpha=.5, edgecolor='w', facecolor='w'),size=25)

# add title and axes labels

plt.show()

### 4. Using KNN for classification

Now that we have familiarised ourselves with the data, let's use the KNN algorithm to predict species lables from the sepal and petal features. 

#### 4.1. Making predictions

__TASK:__ Use ```KNeighborsClassifier()``` from ```sklearn.neighbors``` to first construct a KNN classifier object and then use ```fit()``` on the object to build the classifier from the training set ```X_train```. Finally use ```predict()``` on the classifier object to make predictions for the test dataset ```X_test```.

In [ ]:
# Fitting clasifier to the Training set
# Loading libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score

# Instantiate model (k = 3)


# Fitting the model on X_train


# Predicting the Test set results and store in a list y_pred



#### 4.2. Evaluating predictions

Building confusion matrix using the function ```confusion_matrix()```:

In [ ]:
# compute and print out the confusion matrix
cm = 


Calculating model accuracy using ```accuracy_score()```:

In [ ]:
# compute and print the accuracy of the classifier
accuracy = 
print('\n Accuracy of our model is ' + str(round(accuracy*100, 2)) + '%.')

__TASK:__ Reflect on the above result! Are you happy with this accuracy? Is accuracy indeed the best metric to use here, and if so, why? Repeat __§4.1__ & __§4.3__ with a different $k$ - when do you get better or worse predictions? Do you think the estimated accuracy is reliable?

#### 4.3. Using cross-validation for parameter tuning:

Cross-validation is a technique to systematically split up the training and test sets in different ways so as to obtain more robust performance estimates. We will learn more about cross-validation in the next lecture. For now, let's try to use it to determine the best value of $k$. 

__TASK:__ For a list of values $k\in[1, ..., 50]$ build a KNN classifier using again ```KNeighborsClassifier``` and then use the function ```cross_val_score()``` on ```X_train``` and ```y_train``` to obtain robust estimates of the accuracy of the classifiers with different $k$. Store the results in a list ```cv_scores```.

In [ ]:
# creating list of K for KNN
k_list = list(range(1,50,2))
# creating list of cross validation scores
cv_scores = []

# for each value of k, perform 10-fold cross validation
for k in k_list:
    # instantiate the KNN classifier object with the current value of k

    # compute the accuracy scores for each iteration of the cross-validation using cross_val_score()
    scores = 
    # append the mean of scores to cv_scores using the append() function


__TASK:__ Let's plot the misclassification error, $MSE = 1 - $accuracy.

In [ ]:
# compute the misclassification error from cv_scores
MSE = 

plt.figure()
plt.figure(figsize=(15,10))
# plot the list of k values (k_list) against the corresponding misclassification errors

# add a title and axes labels


sns.set_style("whitegrid")
plt.show()

__REFLECT:__ The best $k$-NN classifier is the one that minimises the misclassification error. Find out what the $k$ is. Now look at the trend of the misclassification error - would you have expected that the error goes up again for higher $k$, and if so why? Google the term 'overfitting'. How can you detect overfitting from the above curve?

In [ ]:
# finding best k
best_k = k_list[MSE.index(min(MSE))]
print("The optimal number of neighbors is %d." % best_k)

__TASK:__ Above we have applied 10-fold cross-validation. Repeat the cross-validation analysis at differen fold-values, changing the ```cv``` property in the ```cross_val_score()``` function. What changes do you see? By searching 'cross-validation' what insights can you get on how to choose the fold-value?

### 5. My own KNN implementation

__For the curious-minded:__ The KNN algorithm is relatively simple. Do you think you can implement your own KNN algorithm?

__Optional task:__ Start by building defining ```class MyKNeighborsClassifier()```, it should include functions ```__init__(self, params)```, ```fit(self, X, y)``` and ```predict(self, X_test)```. The ```predict()``` function will loop over all new data points and make single predictions for each point. For that, implement a separate function ```single_prediction(X, y, x_train, k)```, which computes the distance of all training data points to the new point and considers the class label of the $k$ nearest neighbours for a vote on the predicted class label. 

In [ ]:
import numpy as np
import pandas as pd
import scipy as sp

class MyKNeighborsClassifier():
    """
    My implementation of KNN algorithm.
    """
    
    def __init__(self, n_neighbors=5):
        # initialisation only needs to set a property n_neighbors on self

        
    def fit(self, X, y):
        # X are the features, y the outcomes
        
        """
        Fit the model using X as array of features and y as array of labels.
        """
        n_samples = # set to number of samples
        
        """
        Some error checks
        """
        # number of neighbors can't be larger then number of samples
        if self.n_neighbors > n_samples:
            raise ValueError("Number of neighbors can't be larger then number of samples in training set.")
        
        # X and y need to have the same number of samples
        if X.shape[0] != y.shape[0]:
            raise ValueError("Number of samples in X and y need to be equal.")
        
        """
        Find and save all possible class labels using np.unique()
        """
        
        self.classes_ = # unique class labels
        
        # define the X features and y outcomes
        self.X = # features
        self.y = # outcomes
        
    def predict(self, X_test):
        
        # number of predictions to make and number of features inside single sample using shape on X_test
        n_predictions, n_features = X_test.shape
        
        # allocationg space for array of predictions
        predictions = np.empty(n_predictions, dtype=int)
        
        # loop over all observations
        for i in range(n_predictions):
            # calculation of single prediction
            predictions[i] = single_prediction(self.X, self.y, X_test[i, :], self.n_neighbors)

        return(predictions)

In [ ]:
def single_prediction(X, y, x_test, k):
    
    # number of samples inside training set
    n_samples = X.shape[0]
    
    # create array for distances and targets
    distances = np.empty(n_samples, dtype=np.float64)

    # distance calculation
    for i in range(n_samples):
        distances[i] = # calculate the Euclidian distance between x_test and X using the formula in §1.2
    
    # combining arrays as columns
    distances = sp.c_[distances, y]
    # sorting array by value of first column
    sorted_distances = distances[distances[:,0].argsort()]
    # selecting labels associeted with k smallest distances
    targets = sorted_distances[0:k,1]

    unique, counts = np.unique(targets, return_counts=True)
    return(unique[np.argmax(counts)])

Now instantiate and fit your KNN model on the training set and make predictions for ```X_test``` for a $k$ of your choice.

In [ ]:
# Instantiate learning model (k = 3)
my_classifier = # instantiate MyKNeighborsClassifier()

# Fitting the model with X_train and y_train


# Predicting the Test set results
my_y_pred = # make predictions on X_test

__TASK:__ Compute the accuracy of your KNN classifier and compare to the result above using ```KNeighborsClassifier()```.

In [ ]:
# compute and print the accuracy of the model
accuracy = 
print('Accuracy of our model is equal ' + str(round(accuracy*100, 2)) + ' %.')